# 라이브러리, 데이터 불러오기

In [1]:
!pip install lightfm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 310 kB 8.8 MB/s 
  Created wheel for lightfm: filename=lightfm-1.16-cp37-cp37m-linux_x86_64.whl size=705374 sha256=e84529b158dcc3d1fcd3187ebd8c7749c48dd77769ca532369e0f35d4275f4ed
  Stored in directory: /root/.cache/pip/wheels/f8/56/28/5772a3bd3413d65f03aa452190b00898b680b10028a1021914
Successfully built lightfm


In [4]:
import numpy as np
import pandas as pd
from lightfm.cross_validation import random_train_test_split
from lightfm import LightFM
from lightfm.data import Dataset
from lightfm.evaluation import precision_at_k, auc_score
from scipy.io import mmwrite

In [82]:
ext = pd.read_csv('../3. LOTTO_데이터 및 모델 세이브 파일/clustering/clustering_recommend.csv', index_col=0) #user
data2 = pd.read_csv('../3. LOTTO_데이터 및 모델 세이브 파일/data_preprocessing/data2.csv', index_col=0)
price = pd.read_csv('../3. LOTTO_데이터 및 모델 세이브 파일/new/product_price.csv', index_col=0) #item
cycle = pd.read_csv('../3. LOTTO_데이터 및 모델 세이브 파일/new/상품별 구매주기_0809수정.csv', index_col=0) #item
purchase = pd.read_csv('../3. LOTTO_데이터 및 모델 세이브 파일/new/purchase_list.csv', index_col=0) #score
ext.head()

,cust,연간구매금액2,연간구매금액3,연간구매금액6,연간구매금액236,유통평균주기,제휴평균주기,Lpay평균주기,총합평균주기,Top5상품,...,zon_hlv_Z14,zon_hlv_Z15,zon_hlv_Z16,zon_hlv_Z17,pur6_bin_0,pur6_bin_1,pur6_bin_2,pur6_bin_3,pur6_bin_4,pur6_bin_5
0,M000034966,654120.0,3900.0,0.0,658020.0,56 days 20:00:00,1번 구매,0번 구매,48 days 17:08:34.285714285,['채소' '과자' '조미료' '과일' '속옷/양말/홈웨어'],...,0,0,0,0,1,0,0,0,0,0
1,M000136117,28496031.0,252500.0,1137501.0,28748531.0,12 days 09:55:51.724137931,20 days 22:17:08.571428571,106 days 16:00:00,8 days 04:21:49.090909090,['여성의류' '침구/수예' '패션잡화' '남성의류' '세제/위생'],...,0,0,0,0,0,0,1,0,0,0
2,M000201112,53120.0,0.0,0.0,53120.0,42 days 00:00:00,0번 구매,0번 구매,42 days 00:00:00,['대용식' '과자' '음료' '조리식품' '냉장식품'],...,0,0,0,1,1,0,0,0,0,0
3,M000225114,1937700.0,316200.0,0.0,2253900.0,8 days 05:11:21.081081081,53 days 06:00:00,0번 구매,7 days 05:42:51.428571428,['과자' '냉장식품' '조리식품' '과일' '채소'],...,0,0,0,1,1,0,0,0,0,0
4,M000261625,6052700.0,23250.0,0.0,6075950.0,16 days 00:00:00,1번 구매,0번 구매,16 days 00:00:00,['화장품/뷰티케어' '테넌트/음식점' '퍼스널케어' '모바일' '속옷/양말/홈웨어'],...,0,0,0,1,1,0,0,0,0,0


In [6]:
ext.columns

Index(['cust', '연간구매금액2', '연간구매금액3', '연간구매금액6', '연간구매금액236', '유통평균주기',
       '제휴평균주기', 'Lpay평균주기', '총합평균주기', 'Top5상품', 'Top5상품금액', 'num_pur2',
       'num_pur3', 'num_pur6', 'A01', 'A02', 'A03', 'A04', 'A05', 'B01', 'C01',
       'C02', 'D01', 'D02', 'E01', 'A06', 'L00', 'L01', 'how_many_cop',
       'online_friendly', 'cltv', 'cluster_ae', 'ma_fem_dv_남성', 'ma_fem_dv_여성',
       'ages_20대', 'ages_30대', 'ages_40대', 'ages_50대', 'ages_60대', 'ages_70대',
       'zon_hlv_Z01', 'zon_hlv_Z02', 'zon_hlv_Z03', 'zon_hlv_Z04',
       'zon_hlv_Z05', 'zon_hlv_Z06', 'zon_hlv_Z07', 'zon_hlv_Z08',
       'zon_hlv_Z09', 'zon_hlv_Z10', 'zon_hlv_Z11', 'zon_hlv_Z12',
       'zon_hlv_Z13', 'zon_hlv_Z14', 'zon_hlv_Z15', 'zon_hlv_Z16',
       'zon_hlv_Z17', 'pur6_bin_0', 'pur6_bin_1', 'pur6_bin_2', 'pur6_bin_3',
       'pur6_bin_4', 'pur6_bin_5'],
      dtype='object')

# 데이터 정제

## 유저 데이터

In [7]:
user_meta = ext[['cust', 'cluster_ae','연간구매금액236', '총합평균주기']]
user_meta.loc[user_meta.총합평균주기 == '0번 구매', '총합평균주기'] = '366 days'
user_meta.loc[user_meta.총합평균주기 == '1번 구매', '총합평균주기'] = '365 days'
user_meta.총합평균주기 = user_meta.총합평균주기.apply(lambda x: x.split()[0])
user_meta

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,cust,cluster_ae,연간구매금액236,총합평균주기
0,M000034966,0,658020.0,48
1,M000136117,21,28748531.0,8
2,M000201112,51,53120.0,42
3,M000225114,17,2253900.0,7
4,M000261625,17,6075950.0,16
...,...,...,...,...
29904,M915138788,-1,0.0,366
29905,M931112452,76,0.0,366
29906,M937793665,11,0.0,366
29907,M960942132,11,0.0,366


## 물품 데이터

In [8]:
item_meta = pd.merge(price, cycle[['pd_c', '구매횟수', '구매주기']], on='pd_c', how='left')
item_meta.구매횟수.fillna(1, inplace=True)
item_meta.구매주기.fillna(365, inplace=True)
item_meta

,pd_c,pd_nm,clac_hlv_nm,clac_mcls_nm,buy_am_per_ct,구매횟수,구매주기
0,PD0001,소파,가구,거실가구,2143346,3.548387,173.520661
1,PD0002,스툴/리빙의자,가구,거실가구,43059,2.187500,83.187500
2,PD0003,탁자,가구,거실가구,201200,14.000000,101.185185
3,PD0004,장식장/진열장,가구,거실가구,133424,2.500000,100.416667
4,PD0005,기타가구,가구,기타가구,959597,3.315315,155.100000
...,...,...,...,...,...,...,...
1927,PD1929,여성향수,화장품/뷰티케어,향수,151097,2.812785,128.036263
1928,PD1930,남성향수,화장품/뷰티케어,향수,146227,2.708333,216.079060
1929,PD1931,남녀공용향수,화장품/뷰티케어,향수,142878,2.416058,123.483090
1930,PD1932,남성향수세트,화장품/뷰티케어,향수,197372,2.250000,121.750000


## 구매내역 데이터

In [9]:
purchase.loc[purchase['count']>1, 'count']=1
purchase['count'].value_counts()

1    1637067
Name: count, dtype: int64

In [17]:
purchase.groupby(['cust','pd_c'])['count'].sum().reset_index()[30:60]

,cust,pd_c,count
30,M000034966,PD1393,1
31,M000034966,PD1403,1
32,M000034966,PD1422,1
33,M000034966,PD1437,1
34,M000034966,PD1439,1
35,M000034966,PD1440,1
36,M000034966,PD1484,1
37,M000136117,PD0173,1
38,M000136117,PD0197,1
39,M000136117,PD0290,1


## 벡터화

데이터를 특징벡터화 합니다.

In [18]:
ratings_source = [(purchase['cust'][i], purchase['pd_c'][i]) for i in range(purchase.shape[0])]

user_features_source = [(user_meta['cust'][i],
                         [user_meta['cluster_ae'][i],
                          user_meta['연간구매금액236'][i],
                          user_meta['총합평균주기'][i]]) for i in range(user_meta.shape[0])]

item_features_source = [(item_meta['pd_c'][i],
                        [item_meta['clac_hlv_nm'][i],
                         item_meta['clac_mcls_nm'][i],
                         item_meta['buy_am_per_ct'][i],
                         item_meta['구매주기'][i],
                         item_meta['구매횟수'][i]]) for i in range(item_meta.shape[0])]

In [19]:
dataset = Dataset()
dataset.fit(users=ext['cust'].unique(),
            items=purchase['pd_c'].unique(),
            user_features=user_meta[user_meta.columns[1:]].values.flatten(),
            item_features=item_meta[item_meta.columns[1:]].values.flatten()
            )

In [20]:
interactions, weights = dataset.build_interactions(ratings_source)
item_features = dataset.build_item_features(item_features_source)
user_features = dataset.build_user_features(user_features_source)

train, test = random_train_test_split(interactions, test_percentage=0.1)
train, test = train.tocsr().tocoo(), test.tocsr().tocoo()
train_weights = train.multiply(weights).tocoo()

# 하이퍼파라미터 튜닝
hyperopt를 이용하여 모델에 필요한 하이퍼파라미터를 탐색합니다.

In [19]:
from hyperopt import fmin, hp, tpe, Trials

# Define Search Space
trials = Trials()
space = [hp.choice('no_components', range(20, 200, 10)),
         hp.uniform('learning_rate', 0.01, 1)]

In [20]:
# Define Objective Function
def objective(params):
    no_components, learning_rate = params

    model = LightFM(no_components=no_components,
                    learning_schedule='adagrad',
                    loss='warp',
                    learning_rate=learning_rate,
                    random_state=0)

    model.fit(interactions=train,
              item_features=item_features,
              user_features=user_features,
              sample_weight=train_weights,
              epochs=3,
              verbose=False)

    test_precision = precision_at_k(model, test, k=5, item_features=item_features, user_features=user_features).mean()
    print("no_comp: {}, lrn_rate: {:.5f}, precision: {:.5f}".format(
      no_components, learning_rate, test_precision))
    # test_auc = auc_score(model, test, item_features=item_features).mean()
    output = -test_precision

    if np.abs(output+1) < 0.01 or output < -1.0:
        output = 0.0

    return output

In [21]:
best_params = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=50, trials=trials)

no_comp: 50, lrn_rate: 0.44545, precision: 0.05426
no_comp: 50, lrn_rate: 0.74831, precision: 0.05325
no_comp: 140, lrn_rate: 0.67251, precision: 0.03327
no_comp: 70, lrn_rate: 0.59642, precision: 0.04972
no_comp: 120, lrn_rate: 0.32449, precision: 0.03809
no_comp: 60, lrn_rate: 0.27287, precision: 0.05141
no_comp: 80, lrn_rate: 0.05241, precision: 0.06353
no_comp: 70, lrn_rate: 0.22181, precision: 0.05205
no_comp: 120, lrn_rate: 0.88012, precision: 0.03459
no_comp: 30, lrn_rate: 0.47199, precision: 0.05805
no_comp: 90, lrn_rate: 0.35642, precision: 0.04432
no_comp: 180, lrn_rate: 0.16380, precision: 0.04715
no_comp: 180, lrn_rate: 0.87012, precision: 0.02409
no_comp: 20, lrn_rate: 0.72283, precision: 0.05873
no_comp: 150, lrn_rate: 0.97460, precision: 0.02916
no_comp: 90, lrn_rate: 0.15657, precision: 0.05569
no_comp: 50, lrn_rate: 0.58192, precision: 0.05410
no_comp: 140, lrn_rate: 0.27740, precision: 0.03668
no_comp: 160, lrn_rate: 0.95143, precision: 0.02697
no_comp: 160, lrn_rate:

In [22]:
best_params

{'learning_rate': 0.059548229610706355, 'no_components': 6}

# 모델

In [92]:
# model = LightFM(**best_params, learning_schedule='adagrad', loss='warp', random_state=0)
model = LightFM(loss='warp',
                random_state=2022,
                learning_rate=0.06,
                no_components=100,
                item_alpha=0,
                user_alpha=0)

model.fit(interactions=train,
          item_features=item_features,
          user_features=user_features,
          sample_weight=train_weights,
          epochs=200,
          verbose=True)

# 아이템간 유사도
make_best_items_report(item_embeddings, pd_id, num_search_items=10):
- 아이템간 유사도를 구하는 함수
- item_embeddings : 위에서 구한 아이템의 특징벡터
- pd_id : 유사도를 구하고 싶은 아이템의 인덱스
- num_search_items : 유사도를 구할 아이템의 개수

In [25]:
# Find Similar Items
item_biases, item_embeddings = model.get_item_representations(features=item_features)

def make_best_items_report(item_embeddings, pd_id, num_search_items=10):
    item_id = pd_id - 1

    # Cosine similarity
    scores = item_embeddings.dot(item_embeddings[item_id])  # (10000, )
    item_norms = np.linalg.norm(item_embeddings, axis=1)    # (10000, )
    item_norms[item_norms == 0] = 1e-10
    scores /= item_norms

    # best: score가 제일 높은 item의 id를 num_search_items 개 만큼 가져온다.
    best = np.argpartition(scores, -num_search_items)[-num_search_items:]
    similar_item_id_and_scores = sorted(zip(best, scores[best] / item_norms[item_id]),
                                        key=lambda x: -x[1])

    # Report를 작성할 pandas dataframe
    best_items = pd.DataFrame(columns=['pd_c', 'clac_hlv_nm', 'clac_mcls_nm', 'pd_nm', 'score'])

    for similar_item_id, score in similar_item_id_and_scores:
        pd_c = 'PD'+str(similar_item_id + 1).zfill(4)
        clac_hlv_nm = item_meta[item_meta['pd_c'] == pd_c].values[0][2]
        clac_mcls_nm = item_meta[item_meta['pd_c'] == pd_c].values[0][3]
        pd_nm = item_meta[item_meta['pd_c'] == pd_c].values[0][1]

        row = pd.Series([pd_id, clac_hlv_nm, clac_mcls_nm, pd_nm, score], index=best_items.columns)
        best_items = best_items.append(row, ignore_index=True)

    return best_items

In [26]:
#건해산물-건어물-황태채와 가장 유사한 상품
report01 = make_best_items_report(item_embeddings, 100, 10)
report01

,pd_c,clac_hlv_nm,clac_mcls_nm,pd_nm,score
0,100,건해산물,건어물,황태채,1.000000
1,100,건해산물,건어물,어포채류,0.803241
2,100,계절가전,냉방가전,벽걸이형에어컨,0.392121
3,100,건해산물,건어물,원물천연조미료,0.384578
4,100,공구/안전용품,안전용품,화재안전용품,0.374736
5,100,건해산물,반건생선,과메기,0.283386
6,100,건해산물,건어물,황태/북어류,0.281002
7,100,세제/위생,세탁세제,분말표백제,0.279856
8,100,가구,사무용/학생용가구,책장,0.270245
9,100,냉장/세탁가전,냉장/냉동고,업소용냉장고,0.259879


In [44]:
#채소-한방약재-기타한방약재와 가장 유사한 상품
report02 = make_best_items_report(item_embeddings, 646, 10)
report02

,pd_c,clac_hlv_nm,clac_mcls_nm,pd_nm,score
0,646,속옷/양말/홈웨어,여성양말류,여성캐쥬얼스포츠양말,1.000000
1,646,여성의류,여성의류하의,여성바지,0.841228
2,646,가구,시공/DIY가구,DIY가구,0.760888
3,646,담배,전자담배,전자담배용리필,0.759648
4,646,아웃도어/레저,자전거,MTB,0.734933
5,646,침구/수예,성인침구,성인침구세트,0.711545
6,646,유아동의류,여아특수소재의류,여아피혁의류,0.697828
7,646,채소,두부/콩나물류,두부류,0.692999
8,646,세제/위생,주방세제,일반주방세제,0.679453
9,646,아웃도어/레저,등산,등산화,0.607065


# 추천시스템
- auc score 0.96 달성


In [45]:
test_auc = auc_score(model, test_interactions=test, user_features=user_features, item_features=item_features, num_threads=4).mean()
test_auc

0.96071863

In [85]:
n_items = len(item_meta)
pred = model.predict(15, np.arange(n_items), item_features=item_features, user_features=user_features)
price['pred']=pred
price.sort_values(ascending=False, by='pred')[:10]

,pd_c,pd_nm,clac_hlv_nm,clac_mcls_nm,buy_am_per_ct,pred
452,PD0453,펀치류,문구/사무용품,일반문구/사무용품,8277,-22.138950
320,PD0321,냉동튀김,냉동식품,냉동간편식,7208,-22.682425
319,PD0320,냉동핫도그,냉동식품,냉동간편식,7002,-23.170725
1028,PD1030,플레이세트,완구,실외놀이/스포츠완구,15528,-23.205769
1453,PD1455,약용버섯류,채소,한방약재,5000,-23.245655
1030,PD1032,봉제인형,완구,여아완구,27680,-23.405741
293,PD0294,남성가디건,남성의류,남성의류상의,331312,-23.417711
301,PD0302,남성패딩,남성의류,남성의류아우터,464571,-23.431274
749,PD0751,우렁이,수산물,조개류,4040,-23.483774
1230,PD1232,휴대폰용품,자동차용품,액세서리편의용품,13666,-23.594578


In [89]:
cust_id = 'M757079175'
idx = int(ext[ext.cust==cust_id].index.values)

n_items = len(item_meta)
pred = model.predict(idx, np.arange(n_items),
                     item_features=item_features, user_features=user_features)
price['pred']=pred
price.sort_values(ascending=False, by='pred')[:10]

,pd_c,pd_nm,clac_hlv_nm,clac_mcls_nm,buy_am_per_ct,pred
176,PD0177,딸기,과일,국산과일,10802,-61.539333
192,PD0193,수입포도,과일,수입과일,8669,-62.470448
180,PD0181,수박,과일,국산과일,15625,-63.128998
62,PD0063,홍삼/인삼혼합세트,건강식품,홍삼/인삼가공식품,27098,-63.245487
470,PD0471,카드,문구/사무용품,파티/팬시용품,1109,-63.246033
940,PD0942,현미,양곡,잡곡,12395,-63.281094
237,PD0238,판초콜릿,과자,초콜릿,3104,-63.479809
178,PD0179,무화과,과일,국산과일,11062,-63.481209
718,PD0720,우럭,수산물,어류,5186,-63.490532
138,PD0139,포장이사용품,공구/안전용품,보수용품,3251,-63.508335


In [70]:
pd_cop = data2[['cop_c', 'pd_c']].groupby(['pd_c', 'cop_c'])['cop_c'].count()
pd_cop

pd_c    cop_c
PD0001  A01      298
        A02        1
        A06       14
PD0002  A01        3
        A02      149
                ... 
PD1931  A03        2
        A06       80
PD1932  A01       43
PD1933  A01       35
        A06       13
Name: cop_c, Length: 5835, dtype: int64

**lotto_recommend**(cust_id, pred_cop_c, rec_num=10) : lotto 팀이 구축한 __계열사 확장 기반__ 추천시스템
- cust_id : 롯데의 상품을 추천하고 싶은 유저 id를 입력합니다.
- pred_cop_c : 확장가능성이 높은 계열사를 입력합니다.
- rec_num : 추천할 상품의 개수를 입력합니다.

- 유저에게 추천할 상품이 확장 가능성이 높은 계열사라면 별도로 표기하여 추천에 따른 실질적인 성과을 높입니다.



In [154]:
def lotto_recommend(cust_id, pred_cop_c, rec_num=10):
    n_items = len(item_meta)
    idx = int(ext[ext.cust==cust_id].index.values)
    
    pred = model.predict(idx, np.arange(n_items), item_features=item_features, user_features=user_features)
    price['pred']=pred
    recommend_pd = price.sort_values(ascending=False, by='pred')[:rec_num]
    for product in recommend_pd.pd_c:
        hlv, mcls, item = recommend_pd.loc[recommend_pd.pd_c==product, ['clac_hlv_nm', 'clac_mcls_nm', 'pd_nm']].values[0]
        if pred_cop_c in pd_cop[product].index:
            print(f'"{pred_cop_c}"에서 "{hlv}-{mcls}-{item}"을 사시는 건 어떤가요?')
        else:
            print(f'"{item}"은 어떤가요?')

In [155]:
lotto_recommend('M757079175', 'A01', 15)

"A01"에서 "과일-국산과일-딸기"을 사시는 건 어떤가요?
"A01"에서 "과일-수입과일-수입포도"을 사시는 건 어떤가요?
"A01"에서 "과일-국산과일-수박"을 사시는 건 어떤가요?
"A01"에서 "건강식품-홍삼/인삼가공식품-홍삼/인삼혼합세트"을 사시는 건 어떤가요?
"카드"은 어떤가요?
"현미"은 어떤가요?
"A01"에서 "과자-초콜릿-판초콜릿"을 사시는 건 어떤가요?
"A01"에서 "과일-국산과일-무화과"을 사시는 건 어떤가요?
"우럭"은 어떤가요?
"포장이사용품"은 어떤가요?
"접착제"은 어떤가요?
"야외의자/벤치"은 어떤가요?
"A01"에서 "건강식품-홍삼/인삼가공식품-인삼가공식품"을 사시는 건 어떤가요?
"A01"에서 "인테리어/조명-커튼/블라인드류-커튼"을 사시는 건 어떤가요?
"A01"에서 "주방잡화-주방정리용품/소모품-호일/랩/기름종이"을 사시는 건 어떤가요?


82개로 나뉜 클러스터 중 같은 클러스터에 있는 사람들에게는 비슷한 상품이 추천됩니다.
유저의 구매력을 나타내는 연간구매금액과 구매의 주기를 나타내는 총합평균주기도 고려되어 추천이 가능합니다.

In [165]:
ext[ext.cluster_ae == 29][['cust', '연간구매금액236', '총합평균주기']]

,cust,연간구매금액236,총합평균주기
568,M020881880,143090.0,38 days 04:48:00
1132,M041003585,6352400.0,10 days 16:30:00
1494,M054153739,566108.0,17 days 14:24:00
1503,M054578187,4270156.0,3 days 08:44:51.588785046
1554,M056407939,3519240.0,18 days 00:00:00
...,...,...,...
29051,M695403949,4000.0,1번 구매
29124,M724171938,26500.0,1번 구매
29691,M934474402,13000.0,1번 구매
29740,M952029335,205800.0,32 days 08:00:00


In [174]:
cust_id = 'M041003585'
idx = int(ext[ext.cust==cust_id].index.values)

n_items = len(item_meta)
pred = model.predict(idx, np.arange(n_items),
                     item_features=item_features, user_features=user_features)
price['pred']=pred
price.sort_values(ascending=False, by='pred')[:10]

,pd_c,pd_nm,clac_hlv_nm,clac_mcls_nm,buy_am_per_ct,pred
940,PD0942,현미,양곡,잡곡,12395,-77.710312
180,PD0181,수박,과일,국산과일,15625,-79.205292
211,PD0212,사탕/캔디세트,과자,과자세트,3240,-79.237236
176,PD0177,딸기,과일,국산과일,10802,-79.386368
1263,PD1265,족발,조리식품,온장조리,15503,-79.573212
251,PD0252,기타구기/필드스포츠용품,구기/필드스포츠,기타구기/필드스포츠,31510,-79.659424
56,PD0057,홍삼액,건강식품,홍삼/인삼가공식품,64891,-79.943436
277,PD0278,테니스공,구기/필드스포츠,테니스,4318,-80.108849
702,PD0704,기타회,수산물,생선회,12693,-80.159988
58,PD0059,홍삼절편,건강식품,홍삼/인삼가공식품,56487,-80.236290


In [175]:
cust_id = 'M056407939'
idx = int(ext[ext.cust==cust_id].index.values)

n_items = len(item_meta)
pred = model.predict(idx, np.arange(n_items),
                     item_features=item_features, user_features=user_features)
price['pred']=pred
price.sort_values(ascending=False, by='pred')[:10]

,pd_c,pd_nm,clac_hlv_nm,clac_mcls_nm,buy_am_per_ct,pred
108,PD0109,과메기,건해산물,반건생선,12878,-46.067780
28,PD0029,화장대,가구,침실가구,468064,-46.332474
105,PD0106,반건생선선물세트,건해산물,건해산물선물세트,180000,-46.493351
58,PD0059,홍삼절편,건강식품,홍삼/인삼가공식품,56487,-46.683247
178,PD0179,무화과,과일,국산과일,11062,-46.833328
176,PD0177,딸기,과일,국산과일,10802,-46.981556
14,PD0015,야외의자/벤치,가구,야외용가구,23042,-47.051781
376,PD0377,햄선물세트,냉장식품,햄/소시지,66030,-47.212051
8,PD0009,사무용/학생용가구세트,가구,사무용/학생용가구,1269886,-47.321915
217,PD0218,쿠키,과자,비스킷류,5074,-47.376652


# 실사용예시
구매데이터 중 1년간 1개의 계열사만 이용한 고객들의 데이터를 이용하여 확장 계열사 예측모델로 가장 이용할 가능성이 큰 계열사를 예측하고 그 결과를 바탕으로 상품을 추천하여 실제 확장을 이끌어냅니다.

In [106]:
ext_df = pd.read_csv('../3. LOTTO_데이터 및 모델 세이브 파일/new/only_one_cop.csv', index_col=0) # 1년 동안 한 개의 계열사만 이용한 유저
ext_df.rename(columns={'chnl_dv':'online_prefer'}, inplace=True)
ext_df.head()

,cust,first_cop,br_c,pd_c,pd_nm,clac_hlv_nm,clac_mcls_nm,Lpay,first_cnt,buy_am,...,C02,E01,L00,B01,L01,avg_buy_am,log_avg_buy_am,ma_fem_dv,ages,zon_hlv
0,M000059535,7,1036,6,7,6,6,0,1,46000.0,...,0,0,0,0,0,46000.0,10.736418,1,1,11
1,M000201112,3,615,338,623,56,238,0,20,53120.0,...,0,0,0,0,0,2656.0,7.884953,1,3,16
2,M000350564,0,13,653,628,69,254,0,32,9630400.0,...,0,0,0,0,0,300950.0,12.614703,1,4,9
3,M000658311,0,45,70,35,23,46,0,1,147500.0,...,0,0,0,0,0,147500.0,11.901590,1,0,15
4,M000713279,4,905,141,176,33,206,0,1,39000.0,...,0,0,0,0,0,39000.0,10.571343,1,3,11


In [84]:
import pickle
model = pickle.load(open('../3. LOTTO_데이터 및 모델 세이브 파일/모델 세이브 파일/recommend_model.pickle', 'rb'))

In [107]:
import xgboost as xgb
from xgboost import XGBClassifier
xgb_model = pickle.load(open('../3. LOTTO_데이터 및 모델 세이브 파일/모델 세이브 파일/xgb_model.pickle', 'rb'))

In [140]:
X=ext_df[[ 'A01', 'A02', 'L01', 'A03',
       'C01', 'D02', 'A04', 'D01', 'A05', 'A06', 'E01', 'B01', 'C02', 'L00',
      'online_prefer', 'Lpay', 'pd_c', 'clac_hlv_nm', 'clac_mcls_nm',
       'br_c', 'ma_fem_dv', 'ages', 'zon_hlv', 'cycle_days', 
       'log_avg_buy_am']]
#Y=ext_df.loc[:, 'second_cop']

In [141]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5390 entries, 0 to 5389
Data columns (total 25 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   A01             5390 non-null   int64  
 1   A02             5390 non-null   int64  
 2   L01             5390 non-null   int64  
 3   A03             5390 non-null   int64  
 4   C01             5390 non-null   int64  
 5   D02             5390 non-null   int64  
 6   A04             5390 non-null   int64  
 7   D01             5390 non-null   int64  
 8   A05             5390 non-null   int64  
 9   A06             5390 non-null   int64  
 10  E01             5390 non-null   int64  
 11  B01             5390 non-null   int64  
 12  C02             5390 non-null   int64  
 13  L00             5390 non-null   int64  
 14  online_prefer   5390 non-null   float64
 15  Lpay            5390 non-null   int64  
 16  pd_c            5390 non-null   int64  
 17  clac_hlv_nm     5390 non-null   i

In [125]:
label_dict = {0:'A01', 1:'A02', 2:'A03', 3:'A04', 4:'A05', 5:'A06', 6:'B01', 7:'C01', 8:'C02', 9:'D01', 10:'D02', 11:'E01', 12:'L00', 13:'L01'}

In [164]:
for pred, val in zip(xgb_model.predict(X[5:10]), ext_df.values[5:10]):
    cust_id = val[0]
    pred_cop_c = label_dict[pred]
    print(f'재방문을 환영합니다. {cust_id}님')
    lotto_recommend(cust_id, pred_cop_c, 8)
    print('=-='*20)

재방문을 환영합니다. M000859319님
"A01"에서 "문구/사무용품-일반문구/사무용품-칼/가위"을 사시는 건 어떤가요?
"A01"에서 "시즌스포츠-스키/보드-보드장비"을 사시는 건 어떤가요?
"A01"에서 "유아동의류-유아의류전신-영유아원피스"을 사시는 건 어떤가요?
"독서대"은 어떤가요?
"A01"에서 "조미료-향신료-기타향신료"을 사시는 건 어떤가요?
"포장갈치"은 어떤가요?
"A01"에서 "주류-맥주-국산맥주"을 사시는 건 어떤가요?
"A01"에서 "음료-차음료-전통차음료"을 사시는 건 어떤가요?
=-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-=
재방문을 환영합니다. M000965026님
"사무용/학생용가구세트"은 어떤가요?
"채소즙"은 어떤가요?
"벽걸이형에어컨"은 어떤가요?
"황태채"은 어떤가요?
"기타가구"은 어떤가요?
"책장"은 어떤가요?
"장롱"은 어떤가요?
"발효원액"은 어떤가요?
=-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-=
재방문을 환영합니다. M001086020님
"A01"에서 "과자-비스킷류-크래커"을 사시는 건 어떤가요?
"A01"에서 "남성의류-남성의류아우터-남성정장재킷"을 사시는 건 어떤가요?
"A01"에서 "건해산물-건어물-황태채"을 사시는 건 어떤가요?
"여아가운"은 어떤가요?
"벽걸이형에어컨"은 어떤가요?
"화재안전용품"은 어떤가요?
"A01"에서 "가구-사무용/학생용가구-사무용/학생용가구세트"을 사시는 건 어떤가요?
"A01"에서 "건강식품-건강진액-채소즙"을 사시는 건 어떤가요?
=-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-=
재방문을 환영합니다. M001626218님
"니삭스/오버니삭스"은 어떤가요?
"A01"에서 "아웃도어/레저-캠핑-캠핑침구"을 사시는 건 어떤가요?
"기타"은 어떤가요?
"A01"에서 "건강식품-홍삼/인삼가공식품-인삼